In [1]:
import tensorflow as tf

In [2]:
from keras.applications import MobileNet
image_size = 224
input_shape = (224, 224, 3)
mobile_conn = MobileNet(input_shape=input_shape, include_top=False, weights='imagenet')

Using TensorFlow backend.


17227776/17225924 [==============================] - 6s 0us/step


In [3]:
for layer in mobile_conn.layers[:-6]:
    layer.trainable = False

In [4]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(mobile_conn)

# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              51381248  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 54,612,162
Trainable params: 52,445,186
Non-trainable params: 2,166,976
_________________________________________________________________


In [5]:
data_dir = "/data/ml_data/images/oilwater/"

In [6]:
# Run this to resave all files to avoid issues with incorrect image formats


from PIL import Image
import os

count = 0
for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        count+=1
        cur_file = os.path.join(data_dir, subdir, file)
        print('\r', 'Iteration', count, 'File:', file, end='')
#         print(cur_file)
        if file.endswith(".jpg"):
            try:
                i = Image.open(cur_file)
                i.save(cur_file)
            except:
                os.remove(cur_file)
print(count)


 Iteration 422 File: water_146.jpg422ration 371 File: water_095.jpg


In [7]:
from keras.preprocessing.image import ImageDataGenerator
train_dir = data_dir
validation_dir = data_dir


training_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split = 0.25,
)

# train_datagen = ImageDataGenerator(
#       rescale=1./255,
#       rotation_range=20,
#       width_shift_range=0.2,
#       height_shift_range=0.2,
#       horizontal_flip=True,
#       fill_mode='nearest',
# validation_split = 0.25)

# validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 30
val_batchsize = 10

train_generator = training_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical',
        subset='training')

validation_generator = training_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
        subset='validation')

Found 317 images belonging to 2 classes.
Found 105 images belonging to 2 classes.


In [10]:
output_dir = '/data/md_data/models/'
output_name = 'test_model.h5'

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save(ouput_dir + output_name)

Epoch 1/10
11/10 [===============================] - 93s 8s/step - loss: 5.5965 - acc: 0.6528 - val_loss: 5.5262 - val_acc: 0.6571
Epoch 2/10
11/10 [===============================] - 71s 6s/step - loss: 5.4938 - acc: 0.6592 - val_loss: 5.5262 - val_acc: 0.6571
Epoch 3/10
11/10 [===============================] - 64s 6s/step - loss: 5.5280 - acc: 0.6570 - val_loss: 5.5262 - val_acc: 0.6571
Epoch 4/10
 6/10 [================>.............] - ETA: 24s - loss: 5.7309 - acc: 0.6444

In [8]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [7]:
# import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

K.set_learning_phase(0) # Ignore dropout at inference
keras_model = load_model(output_dir + output_name)
print(keras_model.input)
print(keras_model.output)

builder = saved_model_builder.SavedModelBuilder(output_dir + 'TestModel/1')
signature = predict_signature_def(inputs={"images": keras_model.input},
                                  outputs={"scores": keras_model.output})

# saved_model_export_path = output_dir + 'TestModel/1'
with K.get_session() as sess:
#     tf.saved_model.simple_save(
#         sess,
#         saved_model_export_path,
#         inputs={'input_image': model.input},
#         outputs={t.name:t for t in model.outputs})
    builder.add_meta_graph_and_variables(sess=sess, tags=[tag_constants.SERVING],
                                         signature_def_map={"predict": signature})
builder.save()





# # maybe replace mobile_conn with model
# prediction_signature = tf.saved_model.signature_def_utils.predict_signature_def({"image": mobile_conn.input}, {"prediction":mobile_conn.output})

# # export_path is a directory in which the model will be created
# builder = saved_model_builder.SavedModelBuilder('/development/models/')
# legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

# # Initialize global variables and the model
# init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
# sess.run(init_op)

# # Add the meta_graph and the variables to the builder
# builder.add_meta_graph_and_variables(
#       sess, [tag_constants.SERVING],
#       signature_def_map={
#            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
#                prediction_signature,
#       },
#       legacy_init_op=legacy_init_op)
# # save the graph      
# builder.save()      

Tensor("mobilenet_1.00_224_input_1:0", shape=(?, 224, 224, 3), dtype=float32)
Tensor("dense_2_1/Softmax:0", shape=(?, 2), dtype=float32)
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /development/models/TestModel/1/saved_model.pb


b'/development/models/TestModel/1/saved_model.pb'